 <h1> Joined Assignment Python and Data Analytics </h1>
    
 

<h2> Final Code </h2>


Used Sources

- https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql
- https://kanoki.org/2020/01/21/pandas-dataframe-filter-with-multiple-conditions/
- https://stackoverflow.com/questions/21271581/selecting-pandas-columns-by-dtype
- https://www.datasciencemadesimple.com/strip-space-column-pandas-dataframe-leading-trailing-2/
- https://stackoverflow.com/questions/23208745/adding-dummy-columns-to-the-original-dataframe
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html



In [ ]:
'-------------------------------------------Step 1 - Loading packages------------------------------------------------'

# import needed packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

'-------------------------------------------Step 2 - Loading data----------------------------------------------------'

# filename --> change here for different name
file = './__datasets/Team 3 - US Census Above 50k Predictor.csv'

# creating header names
headers = ['age', 'job', 'type_of_employment', 'working_hours_per_week', 'return_of_investment',
'level_education', 'years_education', 'marital_status', 'relationship_household', 'race',
'gender', 'nationality', 'yearly_income_bigger_50k']

# looking for na_values
na_values = [' ?', '?']

# using pandas to load data
df = pd.read_csv(filepath_or_buffer = file,
                 header             = None,
                 names              = headers,
                 na_values          = na_values,
                 keep_default_na    = True)

# getting rid of leading spaces in object columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

'-------------------------------------------Step 3 - Translate values------------------------------------------------'

# new dataframe for translating the columns
df_translated = pd.DataFrame.copy(df)

# translating the categorical columns into classes
# level of education
all_degrees = {'High_School_Graduate':['HS-grad'], 
               'Bachelors':['Some-college', 'Bachelors', 'Prof-school'],
               'Compulsory':['11th', '10th', '7th-8th', '9th', '12th', '5th-6th', '1st-4th', 'Preschool'],
               'Associate':['Assoc-voc', 'Assoc-acdm'],
               'University':['Masters', 'Doctorate']}

# translating
for degree in all_degrees:
    df_translated.loc[df_translated.level_education.isin(values=all_degrees[degree]), 'level_education'] = degree

# marital status
all_status = {'Single': ['Never-married', 'Widowed', 'Divorced'], 
             'Married': ['Married-spouse-absent', 'Married-civ-spouse', 'Married-AF-spouse', 'Separated']}
        
# translating
for status in all_status:
    df_translated.loc[df_translated.marital_status.isin(values=all_status[status]), 'marital_status'] = status

# relationship households
all_households = {'House_holder':['Wife', 'Husband', 'Unmarried', 'Other-relative', 'Own-child'],
                  'Household_Non_Family':['Not-in-family']}

# translating
for household in all_households:
    df_translated.loc[df_translated.relationship_household.isin(values=all_households[household]), 'relationship_household'] = household

# type of employment
all_employments = {'Private': ['Private'],
                   'Goverment-Employeed': ['Local-gov', 'State-gov', 'Federal-gov'],
                   'Other' : ['Self-emp-not-inc', 'Self-emp-inc', 'Without-pay', 'Never-worked']}

# translating
for employ in all_employments:
    df_translated.loc[df_translated.type_of_employment.isin(values=all_employments[employ]), 'type_of_employment'] = employ

    
# nationlity --> all except USA and NaN to Other
df_translated.loc[(df_translated['nationality'] != 'United-States') & (df_translated['nationality'].notnull()),'nationality'] = 'Other'

# race
all_race = {'White': ['White'],
            'Person_of_Color': ['Black', 'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Other']}

# translating
for race in all_race:
    df_translated.loc[df_translated.race.isin(values=all_race[race]), 'race'] = race

df_translated['race'].value_counts()

# job
all_job = {'Other': ['Other-service'],
            'Manual labor': ['Machine-op-inspct', 'Handlers-cleaners', 'Priv-house-serv', 'Farming-fishing', 'Transport-moving', 'Craft-repair'],
          'Opertional_and_Tech': ['Exec-managerial', 'Sales', 'Prof-specialty', 'Adm-clerical', 'Tech-support'],
          'Security': ['Armed-Forces', 'Protective-serv']}

# translating
for job in all_job:
    df_translated.loc[df_translated.job.isin(values=all_job[job]), 'job'] = job

df_translated['job'].value_counts()

'-------------------------------------------Step 4 - Imputing values-------------------------------------------------'

# declaring all datatypes for later
data_types = {'age': int, 'job':str, 'type_of_employment':str, 'working_hours_per_week':int, 
              'return_of_investment':int, 'level_education':str, 'years_education':int, 'marital_status':str,
              'relationship_household':str, 'race':str, 'gender':str, 'nationality':str, 
              'yearly_income_bigger_50k':str}

#getting all numerical columns
num_cols = []
for col in data_types:
    if data_types[col] == int:
        num_cols.append(col)
        
#getting all categorical columns       
cat_cols = []
for col in data_types:
    if data_types[col] == str:
        cat_cols.append(col)

# numerical columns --> impute with the mean
for col in df_translated[num_cols].columns:
    numerical_mean = int(df_translated[col].mean())
    df_translated[col].fillna(value = numerical_mean,
                             inplace = True)
    
#categorical columns
# these columns we want to change by hand
handmade_columns = ['job', 'nationality']

# changing not handmade columns --> mode
for col in df_translated[cat_cols].drop(labels=handmade_columns, axis=1).columns:
    # getting the mode
    categorical_mode = df_translated[col].mode()[0]
    # filling NAs with Mode
    df_translated[col].fillna(value = categorical_mode,
                             inplace = True) 

# handmade changes
# imputing nationlity using race
dict_nat_race = {}

# for all races get most appearing nationality
for race in df_translated['race'].unique():
    input_value = df_translated[['race', 'nationality']].value_counts()[race].idxmax()
    dict_nat_race[race] = input_value
       
for race in dict_nat_race:
    df_translated.loc[(df_translated['race'] == race) & (df_translated['nationality'].isnull()),'nationality'] = dict_nat_race[race]

# imputing job using education
dict_job_ed = {}

for ed in df_translated['level_education'].unique():
    input_value = df_translated[['level_education', 'job']].value_counts()[ed].idxmax()
    dict_job_ed[ed] = input_value
        
for ed in dict_job_ed:
    df_translated.loc[(df_translated['level_education'] == ed) & (df_translated['job'].isnull()),'job'] = dict_job_ed[ed]
    
'-------------------------------------------Step 5 - Changing Datatypes----------------------------------------------'

# change all datatypes with dictionary
df_translated.astype(dtype = data_types)

'-------------------------------------------Step 6 - Generating Dummy Variables--------------------------------------'

#generating dummy variables for each categorical column, dropping the first column, adding prefix with column name
for col in df_translated.select_dtypes(include=['object']).columns:
    df_translated = pd.concat(objs=[df_translated, 
                                    pd.get_dummies(data=df_translated[col], 
                                                   drop_first=True, 
                                                   prefix = f"""d_{col}""")
                                   ], 
                              axis=1)

'-------------------------------------------Step 7 - Generating Bins-------------------------------------------------'
#creating bins for numerical columns using quartiles/quantiles
for col in df_translated.select_dtypes(include=['int']).columns:
    #cutting data into quartiles
    try:
        df_translated[f'b_{col}'] = pd.qcut(x=df_translated[col], q=4, labels = [0, 1, 2, 3])
    #cutting into quartiles of the same size is not possible
    except:
        # getting the quantile values for the column
        quantiles = np.quantile(df_translated[col].unique(), q=[0, 0.25, 0.5, 0.75, 1])
        quantiles[0] -= 1

        # producing the bins
        df_translated[f'b_{col}'] = pd.cut(x = df_translated[col], bins=quantiles, labels=[0,1,2,3])    

'-------------------------------------------Step 8 - Creating train and test sets------------------------------------'  

# isolating target value into new df
# getting columns for y
if 'd_yearly_income_bigger_50k_>50K' in df_translated.columns:
    y_cols = ['d_yearly_income_bigger_50k_>50K', 'yearly_income_bigger_50k']
elif 'd_yearly_income_bigger_50k_<=50K':
    y_cols = ['d_yearly_income_bigger_50k_<=50K', 'yearly_income_bigger_50k']
else:
    print('Missing column for prediction')
    
y = df_translated.loc[:,y_cols]
# dropping target value from original df
df = df_translated.drop(labels=y_cols, axis=1)


#creating test and train sets using sklearn
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(df_translated, y, test_size=test_size, stratify=y, random_state=1)


X_train.to_excel(excel_writer='train_x.xlsx', sheet_name='train_x')
X_test.to_excel(excel_writer='test_x.xlsx', sheet_name='test_x')
y_train.to_excel(excel_writer='train_y.xlsx', sheet_name='train_y')
y_test.to_excel(excel_writer='test_y.xlsx', sheet_name='test_y')

 <h1> Joined Assignment Python and Data Analytics </h1>
    
 <h2> Loading DataFrame into python </h2>
     
 <h3> Importing packages </h3>

Our start point for preparing the DataFrame to be analized is import the packages pandas (pd), pyplot (plt), numpy (np), seaborn (sns) and train_test_split from sklearn since we will use use pandas to read in and convert the data, pyplot and seaborn to visualize the data, numpy for creating quantiles, and sklearn to split our dataframe into a test and training set. This task will be conducted using Code 2.1.1 in our final code.


In [ ]:
#Code 2.1.1

# importing needed packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

 <h3> Load Data with pandas </h3>
First, we specified a file name. That variable has to be changed whenever the name or saving location changes. After loading it once, we realized there were no headers in the csv. We then specified the column titles in the list <em>headers</em>, see Code 2.1.2.1.
In the next step we checked if we have to convert categorical values into <em>NaN</em>'s. For doing so, we used the following script:

~~~
for col in df.columns:
    print(df[col].value_counts())
~~~

We found out that missing values are represented with a <em> ? </em>. After realizing that the question mark comes with a space, we declared a list <em>na_values</em> for both cases. We then read in the csv using pandas <em>read_csv</em>-function and passing in our <em>headers</em> headers and <em>na_values</em> lists. To check if our convertion went right, we ran the following script knowing that the column <em> job </em> had question marks:

~~~
print(df['job'][df.loc[:, 'job'].isnull() == True ])
~~~

That resulted in a dataframe consisting of 1449 rows showing all missing values of the column. In the next section, we will investigate the quality of the data and deal with these values.
Last but not least we discoverd that there are leading spaces in every categorical column. We got rid of these with the following code:

~~~
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()
~~~

In [ ]:
# Code 2.1.2.1

# Looading data
# filename --> change here for different name
file = './__datasets/Team 3 - US Census Above 50k Predictor.csv'

# creating header names
headers = ['age', 'job', 'type_of_employment', 'working_hours_per_week', 'return_of_investment',
'level_education', 'years_education', 'marital_status', 'relationship_household', 'race',
'gender', 'nationality', 'yearly_income_bigger_50k']

# looking for na_values
na_values = [' ?', '?']

# using pandas to load data
df = pd.read_csv(filepath_or_buffer = file,
                 header             = None,
                 names              = headers,
                 na_values          = na_values,
                 keep_default_na    = True)

# getting rid of leading spaces in object columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()
    

 <h2> Investigating the columns </h2> 
 
 <h3> Frequency of values </h3> 
 
 To get a better understanding of the data we checked the columns. First we analyzed, how many unique values every categorical column has. We found out that there are 14 different jobs, 16 different levels of education and more than 40 nationalities. Creating dummy variables for all those categories will affect the outcome of the model as it will lose degrees of freedom. To evaluate that more, we looked into and plotted the frequency of each categorical column allowing us to make assumptions and create new columns.
Our first approach was cutting all the values under 10% off and put them into the <em> other </em> category. We plotted that on the middle chart, but by doing so we realized that the new category does not provide a rational categorization. 

<h4> Job </h4>
We decided to create four categories according to the type of work: (1) manual labor, (2) operational and tech, (3) security, and (4) other. 

<h4> Type of Employment </h4>
We grouped the types of employment into three categories: (1) government, (2) private, and (3) unemployed and other. The <em>other</em> category includes self employed persons and unpaid employees.

<h4> Level of Education </h4>
We grouped the level of education into five mayor buckets: (1) university, (2) bachelors, (3) associate, (4) high-school-graduate, and (5) compulsory. This categorization was clear post reading reading the <em>Subject Definitions</em> document.

<h4> Marital Status </h4>
We created two categories: (1) single, and (2) married. If the individual had never been married, widowed or divorced, we put them into the single category, else into married.

<h4> Relationship Household </h4>
All persons except the not in family people were grouped into the category householders.

<h4> Race </h4>
We decided to create two categories: (1) white, and (2) other for those who does not identify as white. 

<h4> Nationality </h4>
We decided to create two categories: (1) United States, and (2) other. This was due to the fact that individuals with nationality different from United States were not represented enough to create subcategories for all the unique values found. 

You can see the results of that relabeling in the third chart.

In [ ]:
# Code 2.2.1.1

# new dataframe for translating the columns
df_translated = pd.DataFrame.copy(df)

# translating the categorical columns into classes
# level of education
all_degrees = {'High_School_Graduate':['HS-grad'], 
               'Bachelors':['Some-college', 'Bachelors', 'Prof-school'],
               'Compulsory':['11th', '10th', '7th-8th', '9th', '12th', '5th-6th', '1st-4th', 'Preschool'],
               'Associate':['Assoc-voc', 'Assoc-acdm'],
               'University':['Masters', 'Doctorate']}

# translating
for degree in all_degrees:
    df_translated.loc[df_translated.level_education.isin(values=all_degrees[degree]), 'level_education'] = degree

# marital status
all_status = {'Single': ['Never-married', 'Widowed', 'Divorced'], 
             'Married': ['Married-spouse-absent', 'Married-civ-spouse', 'Married-AF-spouse', 'Separated']}
        
# translating
for status in all_status:
    df_translated.loc[df_translated.marital_status.isin(values=all_status[status]), 'marital_status'] = status

# relationship households
all_households = {'House_holder':['Wife', 'Husband', 'Unmarried', 'Other-relative', 'Own-child'],
                  'Household_Non_Family':['Not-in-family']}

# translating
for household in all_households:
    df_translated.loc[df_translated.relationship_household.isin(values=all_households[household]), 'relationship_household'] = household

# type of employment
all_employments = {'Private': ['Private'],
                   'Goverment-Employeed': ['Local-gov', 'State-gov', 'Federal-gov'],
                   'Other' : ['Self-emp-not-inc', 'Self-emp-inc', 'Without-pay', 'Never-worked']}

# translating
for employ in all_employments:
    df_translated.loc[df_translated.type_of_employment.isin(values=all_employments[employ]), 'type_of_employment'] = employ
    
# nationality --> all except USA and NaN to Other
df_translated.loc[(df_translated['nationality'] != 'United-States') & (df_translated['nationality'].notnull()),'nationality'] = 'Other'

# race
all_race = {'White': ['White'],
            'Person_of_Color': ['Black', 'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Other']}

# translating
for race in all_race:
    df_translated.loc[df_translated.race.isin(values=all_race[race]), 'race'] = race

df_translated['race'].value_counts()

# job
all_job = {'Other': ['Other-service'],
            'Manual labor': ['Machine-op-inspct', 'Handlers-cleaners', 'Priv-house-serv', 'Farming-fishing', 'Transport-moving', 'Craft-repair'],
          'Opertional_and_Tech': ['Exec-managerial', 'Sales', 'Prof-specialty', 'Adm-clerical', 'Tech-support'],
          'Security': ['Armed-Forces', 'Protective-serv']}

# translating
for job in all_job:
    df_translated.loc[df_translated.job.isin(values=all_job[job]), 'job'] = job

df_translated['job'].value_counts()

# create dataframe with all categorical columns for plotting
df_objects = pd.DataFrame.copy(df).select_dtypes(include=['object'])
df_objects_other = pd.DataFrame.copy(df).select_dtypes(include=['object'])



# check how many unique values each column has, plot the different dataframes to compare value count
for col in df_objects.columns:
    
    # counting
    counts = df_objects[col].value_counts()
    print(f"""Column {col}: {counts.count()} different values
Biggest category ({counts.idxmax()}) has {counts.max()} values
Smallest category ({counts.idxmin()}) has {counts.min()} values""")
    
    # imputing other for all values with less than 10% in column frequency
    counts = df_objects_other[col].value_counts()/df_objects[col].count()
    mask = df_objects_other[col].isin(counts[counts<0.1].index)
    df_objects_other.loc[mask,col] = 'other'
    
    # 2 graphs side by side
    fig, ax =plt.subplots(1,3,figsize=(12, 3)) 
    
    # graph 1
    sns.countplot(data = df_objects, x = col, color = 'blue',
             order = df_objects[col].value_counts().index, ax=ax[0])
    ax[0].set_title(f"""{col} original""")
 
    # graph 2
    sns.countplot(data = df_objects_other, x = col, color = 'red',
             order = df_objects_other[col].value_counts().index, ax=ax[1])
    ax[1].set_title(f"""{col} manipulated""")
    
    # graph 3
    sns.countplot(data = df_translated, x = col, color = 'green',
             order = df_translated[col].value_counts().index, ax=ax[2])
    ax[2].set_title(f"""{col} translated""")
    
    for ax in fig.axes:
        plt.sca(ax)
        plt.xticks(rotation=90)
        ax.set_ylabel('')
        ax.set_xlabel('')

    plt.show() 


<h3> Missing values </h3> 

<h4> Detecting missing values </h4> 
Next we analysed the missing values, and found out that only three columns contained the 3394 missing values using the following code:

~~~
print(df.isnull().sum().sum())
~~~

So we looked deeper into the columns <em>job</em>, <em> type_of_employment </em> and <em> nationality </em> leading us to the following numbers: 

~~~
                    count_nas  percent_nas
job                      1449         5.12
type_of_employment       1442         5.10
nationality               503         1.78
~~~

This reveals that 5% of the values are contained in the columns <em>job</em>, and <em>type_of_employment</em>. We  investigated these missing values by creating flagging columns, see Code 2.2.1.1, in order to identify any patterns. The analysis revealed that 26,348 rows includes missing values, and 18 of those rows had 3 missing values per row.

~~~
Count of missing values per row
0    26348
1      492
2     1424
3       18
~~~

To avoid loosing the existing data included in the rows with missing values we decided to impute the missing values.

In [ ]:
# Code 2.2.1.1

#creating empty dataframe
values_df= pd.DataFrame()   

# adding count of missing values and missing percentage for every column   
values_df['count_nas'] = df_translated.isnull().sum(axis=0)
values_df['percent_nas'] = (df_translated.isnull().mean(axis=0)*100).round(decimals=2)

# printing how many missing values for each column
print(f"""\nmissing values for each column:
{values_df.loc[:,:][values_df.loc[:, 'count_nas']>0]}\n""")

# creating flagging columns
DATAFRAME = df_translated

# developing a loop to automatically flag missing values
for col in DATAFRAME:

    if DATAFRAME[col].isnull().astype(int).sum() > 0:
        DATAFRAME['m_'+col] = DATAFRAME[col].isnull().astype(int)

df_translated['m_sum'] = df_translated['m_job'] + df_translated['m_type_of_employment'] + df_translated['m_nationality']

#printing summary of missing values
print(df_translated['m_sum'].value_counts().sort_index())



<h4> Imputing values </h4> 

Our imputation strategy was based on the type of the data; since all missing values were categorical, we decided to input the mode, however, for job and nationality we went with a slightly different strategy. 

We discovered that the type of job was highly dependent on the level of education, thus imputing the mode of job would be inefficient so, we decided to impute the mode based on level of education.

Additionally, we decided impute missing values in nationality based on race.

Finally, had we encountered any missing numerical values, we decided to impute the mean.

In [ ]:
# Code 2.2.2.1
df_translated[['level_education', 'job']].value_counts().sort_index()['Associate']


In [ ]:
# Code 2.2.2.2
df_translated[['level_education', 'job']].value_counts().sort_index()['University']


In [ ]:
# Code 2.2.2.3

# numerical columns --> impute with the mean
for col in df_translated[num_cols].columns:
    numerical_mean = int(df_translated[col].mean())
    df_translated[col].fillna(value = numerical_mean,
                             inplace = True)
    
#categorical columns
# these columns we want to change by hand
handmade_columns = ['job', 'nationality']

# changing not handmade columns --> mode
for col in df_translated[cat_cols].drop(labels=handmade_columns, axis=1).columns:
    # getting the mode
    categorical_mode = df_translated[col].mode()[0]
    # filling NAs with Mode
    df_translated[col].fillna(value = categorical_mode,
                             inplace = True) 

# handmade changes
# imputing nationlity using race
dict_nat_race = {}

# for all races get most appearing nationality
for race in df_translated['race'].unique():
    input_value = df_translated[['race', 'nationality']].value_counts()[race].idxmax()
    dict_nat_race[race] = input_value
       
for race in dict_nat_race:
    df_translated.loc[(df_translated['race'] == race) & (df_translated['nationality'].isnull()),'nationality'] = dict_nat_race[race]

# imputing job using education
dict_job_ed = {}

for ed in df_translated['level_education'].unique():
    input_value = df_translated[['level_education', 'job']].value_counts()[ed].idxmax()
    dict_job_ed[ed] = input_value
        
for ed in dict_job_ed:
    df_translated.loc[(df_translated['level_education'] == ed) & (df_translated['job'].isnull()),'job'] = dict_job_ed[ed]
    

 <h2> Creating Dummy-variables for categorical columns</h2>
 
 To compare the impact of the summarization from values to <em> other </em> we created dummy variables for both, the new and the original dataframe. 

In [ ]:
# Code 2.3.1

# creating dummy variables for categorical columns  
for col in df_translated.select_dtypes(include=['object']).columns:
    df_translated = pd.concat(objs=[df_translated, pd.get_dummies(data       = df_translated[col], 
                                                                  drop_first = True, 
                                                                  prefix     = f"""d_{col}""")
                               ], 
                          axis=1)

 <h2> Creating Bins for numerical columns </h2>
 
Since the Naives Bayes Classification only works with categorical values, we translated our numerical columns into bins. This was done by splitting the columns into 4 categories of the same size. 
However, this method did not work in all cases, for example return of investment as it has a lot of zeros and so it was not possible to split into quantiles, hence we decided to split by unique values, see Code 2.4.1.

In [ ]:
#Code 2.4.1

# example for return of investment --> jumps into except case
try:
    df_translated['return_of_investment'] = pd.qcut(x=df_translated['return_of_investment'], q=4, labels = [0, 1, 2, 3])
except:
    # get the quantile values for the column
    quantiles = np.quantile(df_translated['return_of_investment'].unique(), q=[0, 0.25, 0.5, 0.75, 1])

    # produce the bins
    df_translated['RoI_Bins'] = pd.cut(x = df_translated['return_of_investment'], bins=quantiles, labels=[0,1,2,3])

 <h2> Create train and test sets </h2>
 
 The first step in creating the sets was to isolate the target value, <em> yearly_income_bigger_50k </em>. We did that by slicing our dataframe, and then we dropped that column from the original dataframe.
 To create the sets, we used the train-test-split module from the sklearn library, stratifying the data with the target variable ensuring that both test and train sets included income higher and lower than $50k.
 

In [ ]:
# Code 2.5.1

# isolating target value into new df
# getting columns for y
if 'd_yearly_income_bigger_50k_>50K' in df_translated.columns:
    y_cols = ['d_yearly_income_bigger_50k_>50K', 'yearly_income_bigger_50k']
elif 'd_yearly_income_bigger_50k_<=50K':
    y_cols = ['d_yearly_income_bigger_50k_<=50K', 'yearly_income_bigger_50k']
else:
    print('Missing column for prediction')
    
y = df_translated.loc[:,y_cols]
# dropping target value from original df
df = df_translated.drop(labels=y_cols, axis=1)


#creating test and train sets using sklearn
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(df_translated, y, test_size=test_size, stratify=y, random_state=1)


X_train.to_excel(excel_writer='train_x.xlsx', sheet_name='train_x')
X_test.to_excel(excel_writer='test_x.xlsx', sheet_name='test_x')
y_train.to_excel(excel_writer='train_y.xlsx', sheet_name='train_y')
y_test.to_excel(excel_writer='test_y.xlsx', sheet_name='test_y')